# Movie Success Predictor
## Notebook 02: Data Cleaning & Preparation

### Objective
In this notebook, we:
- Merge the movies and credits datasets
- Clean and prepare the data for analysis and modeling
- Handle missing values and inconsistencies
- Create a processed dataset for later phases

## Problem Context

This project focuses on predicting movie success through two modeling tasks:

1. **Hit / Flop Classification**
   - Defined using Return on Investment (ROI)

2. **IMDb Rating Prediction**
   - Predicting movie ratings (`vote_average`)

The goal of this phase is to clean and prepare the raw data so that it can
reliably support exploratory analysis and predictive modeling in later phases.


## Step 1: Import Required Libraries

In this step, we import all necessary libraries required for data manipulation,
numerical operations, and warning handling.


In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully.")


Libraries imported successfully.


## Step 2: Load Raw TMDB Datasets

We load the original movies and credits datasets from the raw data directory.
These datasets will be merged and cleaned in subsequent steps.


In [3]:
movies = pd.read_csv("../data/raw/tmdb_5000_movies.csv")
credits = pd.read_csv("../data/raw/tmdb_5000_credits.csv")

print("Dataframes loaded successfully.")


Dataframes loaded successfully.


## Step 3: Merge Movies and Credits Datasets

The movies and credits datasets contain complementary information.
They are merged using a common movie identifier.


In [4]:
df = movies.merge(
    credits,
    left_on="id",
    right_on="movie_id",
    how="inner"
)
print("Dataframes merged successfully.")
print(f"Merged dataframe shape: rowns={df.shape[0]}, columns={df.shape[1]}")

Dataframes merged successfully.
Merged dataframe shape: rowns=4803, columns=24


## Step 4: Post-Merge Validation

In this step, we validate the merged dataset to ensure:
- No unexpected row duplication occurred
- Key identifier columns are consistent
- The merge operation was successful


In [5]:
print("Duplicated rows found.",df.duplicated().sum())
print((df['id'] == df['movie_id']).all()) # Verify that 'id' and 'movie_id' columns are identical for all rows

Duplicated rows found. 0
True


## Step 5: Remove Redundant Columns and Standardize Naming

After merging, some columns represent the same information
(e.g., movie title and movie ID).

In this step, we remove redundant columns and standardize column names
to maintain a clean dataset.


In [6]:
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
df=df.drop(columns=['movie_id','title_y'])
print("Redundant columns removed.")
df=df.rename(columns={'title_x':'title'})
print("Columns renamed successfully.")
df.head(1)

Redundant columns removed.
Columns renamed successfully.


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


## Step 6: Missing Value Analysis

In this step, we analyze missing values in the merged dataset.

This helps determine which columns require cleaning or special handling
in later steps.


In [8]:
missing_count=df.isna().sum()
missing_percent=(missing_count/len(df)*100).round(2)
missing_data=pd.DataFrame({
    'Columns':missing_count.index,
    'Missing_Count':missing_count.values,
    'Missing_Percentage':missing_percent.values
})
missing_data=missing_data[missing_data['Missing_Count']>0]
missing_data.sort_values(by='Missing_Percentage',ascending=False)

,Columns,Missing_Count,Missing_Percentage
2,homepage,3091,64.36
16,tagline,844,17.57
7,overview,3,0.06
13,runtime,2,0.04
11,release_date,1,0.02


## Step 7: Missing Value Handling Strategy

Based on the missing value analysis, we define a column-wise strategy
for handling missing data. No modifications are applied in this step.

### Missing Value Decisions

**1. homepage (64.36% missing)**
- Very high percentage of missing values.
- Not critical for predicting movie success or rating.
- **Decision:** Drop this column.

**2. tagline (17.57% missing)**
- Textual feature with moderate missing values.
- Limited predictive value for this project.
- **Decision:** Drop this column.

**3. overview (0.06% missing)**
- Important text description of the movie.
- Very few missing values.
- **Decision:** Keep and fill missing values if required.

**4. runtime (0.04% missing)**
- Important numerical feature.
- Missing values are negligible.
- **Decision:** Keep and fill missing values using appropriate statistics.

**5. release_date (0.02% missing)**
- Important for extracting release month/season.
- Missing values are negligible.
- **Decision:** Keep and handle missing values during date processing.


## Step 8: Apply Missing Value Handling

In this step, we apply the missing value handling decisions
defined in the previous step. Only justified operations are performed.


In [9]:
df=df.drop(columns=["homepage","tagline"])
print("Columns with high missing values dropped to reduce noise.")

Columns with high missing values dropped to reduce noise.


In [10]:
df["overview"]=df["overview"].fillna("No overview available.")
print("Missing values in 'overview' filled with default text.")
df["runtime"]=df["runtime"].fillna(df["runtime"].median())
print("Missing values in 'runtime' filled with median value to avoid influence of outliers.")

Missing values in 'overview' filled with default text.
Missing values in 'runtime' filled with median value to avoid influence of outliers.


## Step 9: Release Date Processing

In this step, we convert the release date into datetime format and
extract meaningful temporal features such as year, month, and season.


In [11]:
df["release_date"]=pd.to_datetime(df["release_date"],errors="coerce")
print("Converted 'release_date' to datetime format.")
print()
# Fill missing release_date with median date
median_release_date=df["release_date"].median()
df["release_date"]=df["release_date"].fillna(median_release_date)
print("Missing values in 'release_date' filled with median date.")
print()
# Extract year, month, day from release_date
df["release_year"]=df["release_date"].dt.year
print("Extracted 'release_year' from 'release_date'.")
df["release_month"]=df["release_date"].dt.month
print("Extracted 'release_month' from 'release_date'.")
df["release_day"]=df["release_date"].dt.day
print("Extracted 'release_day' from 'release_date'.")


def get_release_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return np.nan
df['release_season'] = df['release_month'].apply(get_release_season)
print("Derived 'release_season' from 'release_month'.")
df.head(1)

Converted 'release_date' to datetime format.

Missing values in 'release_date' filled with median date.

Extracted 'release_year' from 'release_date'.
Extracted 'release_month' from 'release_date'.
Extracted 'release_day' from 'release_date'.
Derived 'release_season' from 'release_month'.


,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,cast,crew,release_year,release_month,release_day,release_season
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009,12,10,Winter


- Release date is converted to datetime format for safe processing.
- Missing release dates are filled using the median date.
- Temporal features such as year, month, day, and season are extracted
  to capture time-based patterns in movie performance.


In [12]:
df["status"].value_counts()

status
Released           4795
Rumored               5
Post Production       3
Name: count, dtype: int64

**As we can see our 99.7% of data is 'Released', we can drop this column as it does not provide any useful information for our model because we have very little variation in this feature and we can conclude that we are doing only for released movies because we cant predict the success of unreleased movies.**

## Step 10: Handle Movie Status and Remove Redundant Columns

Movies that are not yet released do not have reliable revenue or rating data.
Therefore, we first filter the dataset to include only released movies.
After filtering, the status column becomes redundant and is removed.

In [13]:
df = df[df['status'] == 'Released']
print("Filtered dataset to include only released movies.") #only released movies have reliable revenue and rating data
df = df.drop(
    columns=[
        'status',
        'release_date',
        'overview',
        'original_title'
    ]
)
print("Removed 'status', 'release_date', 'overview', and 'original_title' columns as they are no longer needed.")
print(f"Final dataframe shape: rows={df.shape[0]}, columns={df.shape[1]}")


Filtered dataset to include only released movies.
Removed 'status', 'release_date', 'overview', and 'original_title' columns as they are no longer needed.
Final dataframe shape: rows=4795, columns=20


## Step 11A: Verify Director Uniqueness

Before extracting the director feature, we verify whether each movie
contains exactly one director entry in the crew data.


In [14]:
import ast

def count_directors(crew_str):
    crew_list = ast.literal_eval(crew_str)
    return sum(1 for member in crew_list if member.get('job') == 'Director')

df['director_count'] = df['crew'].apply(count_directors)
df['director_count'].value_counts().sort_index()

director_count
0       28
1     4459
2      277
3       21
4        3
6        2
7        1
8        1
12       2
21       1
Name: count, dtype: int64

### Director Feature Handling Decision

Director information is not perfectly uniform across all movies:
- Most movies have exactly one director.
- Some movies have multiple directors.
- A very small number have no director listed.

To handle this consistently:
- If one or more directors are present, the first listed director is selected.
- If no director is present, the value is set to "Unknown".

This approach balances data completeness and simplicity
while avoiding arbitrary assumptions.


## Step 11B: Verify Producer Distribution

Before deciding whether to include producer information,
we analyze how many producers are associated with each movie.


In [15]:
import ast

def count_producers(crew_str):
    crew_list = ast.literal_eval(crew_str)
    return sum(
        1 for member in crew_list
        if member.get('job') in ['Producer', 'Executive Producer']
    )

df['producer_count'] = df['crew'].apply(count_producers)
df['producer_count'].value_counts().sort_index()


producer_count
0     954
1     529
2     651
3     676
4     530
5     442
6     340
7     197
8     176
9      90
10     63
11     51
12     32
13     20
14     13
15      9
16      9
17      6
18      1
19      2
20      1
21      1
26      2
Name: count, dtype: int64

### Producer Feature Decision

Producer data shows high variability across movies, with many films having
multiple producers and no clear primary producer. The distribution is highly
skewed and ambiguous.

To avoid introducing noise and arbitrary assumptions:
- Producer information is excluded from this project version.
- This decision is based on empirical analysis of producer counts per movie.


## Step 11C: Extract Genres and Director

In this step, we extract structured features from JSON-like columns:
- Genres are extracted as a list of genre names.
- Director is extracted from crew data using predefined rules.

No encoding or transformation is performed in this step.


In [16]:
def extract_genres(genres_str):
    genres_list=ast.literal_eval(genres_str)
    return [genres["name"] for genres in genres_list]
df['genres_list']=df['genres'].apply(extract_genres)
print("Extracted genres into a list.")

Extracted genres into a list.


- The `genres` column contains a JSON-like string.
- `ast.literal_eval()` safely converts it into a Python list.
- Each genre dictionary contains a `name` field.
- A new column `genres_list` stores genre names as a list.


In [17]:
def extract_director(crew_str):
    crew_list = ast.literal_eval(crew_str)
    
    for member in crew_list:
        if member.get('job') == 'Director':
            return member.get('name')
    
    return 'Unknown'
df['director'] = df['crew'].apply(extract_director)
print("Extracted director names from crew data.")

Extracted director names from crew data.


In [18]:
df.head(1)

,budget,genres,id,keywords,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,title,vote_average,vote_count,cast,crew,release_year,release_month,release_day,release_season,director_count,producer_count,genres_list,director
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009,12,10,Winter,1,3,"[Action, Adventure, Fantasy, Science Fiction]",James Cameron


- Crew data is parsed into a list of dictionaries.
- The first occurrence of a Director is selected.
- If no Director is found, the value is set to "Unknown".
- This logic ensures consistency across all movies.


## Step 11D: Remove Raw JSON and Helper Columns

After extracting the required information from JSON-like columns,
we remove the original raw columns and any temporary helper columns
to keep the dataset clean and ready for modeling.


In [19]:
df = df.drop(
    columns=[
        'genres',
        'keywords',
        'production_companies',
        'production_countries',
        'spoken_languages',
        'cast',
        'crew',
        'director_count',
        'producer_count'
    ],
    errors='ignore'
)
print("Dropped original string columns after extraction of relevant information.")
print(f"Cleaned dataframe shape: rows={df.shape[0]}, columns={df.shape[1]}")

Dropped original string columns after extraction of relevant information.
Cleaned dataframe shape: rows=4795, columns=15


## Step 12: Remove Movies with Invalid Core Values

Movies with zero budget, revenue, or runtime do not represent valid
business cases and prevent reliable feature engineering.
Such rows are removed to preserve data integrity.


In [20]:
print("movie with 0 budget:", (df["budget"]==0).sum())
print("movie with 0 revenue:", (df["revenue"]==0).sum())
print("movie with 0 runtime:", (df["runtime"]==0).sum())
print("movie with 0 budget 0 revenue", ((df["budget"]==0) & (df["revenue"]==0)).sum())


movie with 0 budget: 1032
movie with 0 revenue: 1420
movie with 0 runtime: 34
movie with 0 budget 0 revenue 885


In [21]:
df = df[
    (df['budget'] > 0) &
    (df['revenue'] > 0) &
    (df['runtime'] > 0)
].copy()
print("Removed movies with zero budget, revenue, or runtime for data integrity.")
print(f"Dataframe shape after removing zero budget/revenue/runtime: rows={df.shape[0]}, columns={df.shape[1]}")

Removed movies with zero budget, revenue, or runtime for data integrity.
Dataframe shape after removing zero budget/revenue/runtime: rows=3228, columns=15


- Zero values in budget, revenue, or runtime represent missing or invalid data.
- These fields are essential for ROI and profit-based features.
- Removing such rows ensures realistic and trustworthy modeling.


## Step 13: Save Processed Dataset

In [22]:
df.to_csv("../data/processed/processed_movies.csv", index=False)
print("Processed dataset saved successfully to data/processed/processed_movies.csv")

Processed dataset saved successfully to data/processed/processed_movies.csv


In [23]:
type(df['genres_list'].iloc[0])

list

## Phase 2 Completion Summary

In this phase, we:
- Merged movies and credits datasets
- Cleaned missing values with justified strategies
- Filtered to released movies only
- Removed rows with invalid core business values (budget, revenue, runtime)
- Extracted structured features (genres_list, director)
- Removed noisy and redundant columns
- Produced a clean, model-ready dataset

The processed dataset has been saved to `data/processed/clean_movies.csv`
and is ready for exploratory data analysis and modeling.
